In [ ]:
"""
    Instruções:
        [v]- Download dos dados municipais do estado de Minas Gerais;
        [v]- Transformação dos dados de desmatamento;
        [v] Processamento dos dados:
            -reprojeção para EPSG:31983;
            -cálculo de área;
        [v] Geração dos arquivos de saída no formato GeoJSON;
"""

In [2]:
import geopandas as gpd
import pandas as pd
import requests
from bs4 import BeautifulSoup

def obter_municipios_geojson(url, output_file):
    municipios = gpd.read_file(url)
    municipios = municipios.to_crs(epsg=31983)
    municipios["area_km2"] = municipios['geometry'].area / 10**6
    municipios.to_file(output_file, driver="GeoJSON")

def obter_dados_wikipedia(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        tabela = soup.find('table')
        dados = pd.read_html(str(tabela), flavor='html5lib')[0]
        return dados
    else:
        print(f"Falha na requisição. Status code: {response.status_code}")
        return None

def combinar_dados(dados_geojson, dados1, dados2, output_file):
    dados1.columns = ['Posição', 'Município', 'Produto Interno Bruto Nominal (2020)', 'PIB Per Capita (2020)']
    dados2.columns = ['Posição', 'Município', 'População']

    dados_combinados = pd.merge(dados_geojson, dados1, left_on='name', right_on='Município', how='left')
    dados_combinados = pd.merge(dados_combinados, dados2, on='Município', how='left')

    dados_combinados.to_csv(output_file, index=False)
    print("Dados combinados e salvos com sucesso!")

url_geojson_municipios = "https://raw.githubusercontent.com/tbrugz/geodata-br/master/geojson/geojs-31-mun.json"
output_geojson_municipios = "dados/municipios-mg.geojson"
url_pib = "https://pt.wikipedia.org/wiki/Lista_de_munic%C3%ADpios_de_Minas_Gerais_por_PIB"
url_populacao = "https://pt.wikipedia.org/wiki/Lista_de_munic%C3%ADpios_de_Minas_Gerais_por_popula%C3%A7%C3%A3o"
output_csv_combinado = "dados/populacao_pib_municipios_mg.csv"

obter_municipios_geojson(url_geojson_municipios, output_geojson_municipios)
dados_pib = obter_dados_wikipedia(url_pib)
dados_populacao = obter_dados_wikipedia(url_populacao)

municipios_geojson = gpd.read_file(output_geojson_municipios)

if dados_pib is not None and dados_populacao is not None:
    combinar_dados(municipios_geojson, dados_pib, dados_populacao, output_csv_combinado)


Dados combinados e salvos com sucesso!
